In [1]:
from functools import partial

from sepal_ui import widgetFactory as wf
from sepal_ui import widgetBinding as wb
from sepal_ui.scripts import utils

from scripts import run
from utils import messages as ms

In [2]:
#use a class to define your input and output in order to have mutable variables
class Gfc_export_IO:
    def __init__(self):
        
        #set up output
        self.clip_map = None
        
gfc_export_io = Gfc_export_IO()

In [3]:
#create the tile with the links
id_ = "gfc_export_widget"
title = "Results"
    
gfc_results = wf.Tile(id_, title, inputs=[''])

In [4]:
import ipyvuetify as v

#create the output alert 
gfc_export_output = wf.OutputWidget(ms.GFC_EXPORT)

#create the button that start your process
gfc_export_btn = wf.ProcessBtn('Export data')

#create a process tile
id_ = "gfc_export_widget"
title = 'Export the data'

gfc_export = wf.Tile(
    id_, 
    title, 
    btn=gfc_export_btn,  
    output=gfc_export_output,
    inputs=['']
)

#bind the button to the process by writing a custom function
def process_start(widget, event, data, output, gfc_results):
    
    #debug
    #asset = 'users/bornToBeAlive/aoi_PU'
    #threshold = 20
    #visualization = True
    
    asset = gfc_aoi_IO.assetId
    threshold = gfc_viz_io.threshold
    visualization = gfc_viz_io.visualization
    
    #toggle the loading button
    utils.toggleLoading(widget)
    
    #check inputs
    if not wb.checkInput(asset, output, ms.NO_AOI): return utils.toggleLoading(widget)
    if not wb.checkInput(threshold, output, ms.NO_THRESHOLD): return utils.toggleLoading(widget)
    if not wb.checkInput(visualization, output, ms.NO_VIZ): return utils.toggleLoading(widget)
    
    #retreive the data
    #launch any process you want
    clip_map, csv_file = run.gfcExport(asset, threshold, output)
    
    #validate the visualization process
    setattr(gfc_export_io, 'clip_map', clip_map)
    
    #display the results 
    result_layout = run.displayGfcResults(asset, threshold, output)
    gfc_results.children[0].children = [gfc_results.children[0].children[0]] + result_layout
    
    #create the links
    gfc_download_csv = wf.DownloadBtn('csv', path=csv_file)
    gfc_download_tif = wf.DownloadBtn('raster', path=clip_map)
    
    #replace the disabled buttons links 
    gfc_results.children[0].children[1].children[0].href = gfc_download_csv.href
    gfc_results.children[0].children[1].children[1].href = gfc_download_tif.href
    
    #enable them
    gfc_results.children[0].children[1].children[0].disabled = False
    gfc_results.children[0].children[1].children[1].disabled = False
    
    #toggle the loading button
    return utils.toggleLoading(widget)

gfc_export_btn.on_event('click', partial(
    process_start, 
    output = gfc_export_output,
    gfc_results = gfc_results
))

In [5]:
#this tiles will only be displayed if you launch voila from this file 
gfc_export

Layout(align_center=True, children=[Card(children=[Html(children=['Export the data'], tag='h2'), Flex(children…

In [6]:
#this tiles will only be displayed if you launch voila from this file 
gfc_results

Layout(align_center=True, children=[Card(children=[Html(children=['Results'], tag='h2'), Flex(children=[Layout…